In [193]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/auto-grading-data/Traing_Data_analysis.xlsx
/kaggle/input/auto-grading-data/Traing_Data_614.xlsx
/kaggle/input/auto-grading-data/Traing_Data_translated.xlsx
/kaggle/input/auto-grading-data/Test_Dataset.xlsx


In [194]:
!pip install spacy  
!pip install -U sentence-transformers
!python -m spacy download en_core_web_md
import pandas as pd
import numpy as np
import gensim
import torch
import spacy
import fasttext.util
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from scipy.sparse import vstack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 24.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [195]:
test_data = pd.read_excel("/kaggle/input/auto-grading-data/Test_Dataset.xlsx")
train_data_644= pd.read_excel("/kaggle/input/auto-grading-data/Traing_Data_614.xlsx")
train_data_analysis = pd.read_excel("/kaggle/input/auto-grading-data/Traing_Data_analysis.xlsx")
train_data_translated = pd.read_excel("/kaggle/input/auto-grading-data/Traing_Data_translated.xlsx")

In [196]:
import gensim.downloader as api
# Load Word2Vec model (pre-trained model available in gensim)
word2vec_model = api.load("word2vec-google-news-300") 

In [197]:
# Load spaCy with GloVe vectors
nlp = spacy.load("en_core_web_md") 

In [198]:
def get_embeddings(model, text):
    words = text.split()  # Split the text into individual words
    word_embeddings = []
    for word in words:
        if word in model:  # Check if the word is in the Word2Vec vocabulary
            word_embeddings.append(model[word])  # Retrieve the word embedding
        else:
            # Handle out-of-vocabulary words if needed
            pass  # For example, you might skip the word or use a default embedding
    
    if word_embeddings:
        # Calculate the average of word embeddings or apply other aggregation methods
        sentence_embedding = np.mean(word_embeddings, axis=0)  # Calculate the mean along the axis

        return sentence_embedding  # Return the sentence-level embedding
    else:
        return np.zeros(model.vector_size)  # Return zero vector if no embeddings were found


In [199]:
print(test_data.columns)

Index(['question', 'model_answer', 'student_answer', 'grade', 'source', 'ID'], dtype='object')


In [200]:
def Gradanizer(num, grade_mode="Fair"):
    """
    This function take the garde form 0 to 1 and output that garde in form 0 to 5

    parameters
    num: the garde in form 0 to 1
    grade_mode: how fair you want to model to be [fair: the exact transformation with out any lose ranges, easy: more skewed into higher grades more 
    common, lose_ends: more skewed into higher grades and lower grades]
    """
    def get_region_value_f(number):    
        intervals = [
        (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
        (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
        (2.7273, 3.1818), (3.1818, 3.6364), (3.6364, 4.0909),
        (4.0909, 4.5455), (4.5455, 6)]
        values=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]
        
    def get_region_value_e(number):    
        intervals = [
        (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
        (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
        (2.7273, 3.1818), (3.1818, 3.6364)]
        values=[0,0.5,1,1.5,2,2.5,3,3.5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]
    def get_region_value_l(number):    
        intervals = [
        (1.0000, 1.3571), (1.3571, 1.7143), (1.7143, 2.0714),
        (2.0714, 2.4286), (2.4286, 2.7857), (2.7857, 3.1429),
        (3.1429, 3.5000)]
        values=[0.5,1,1.5,2,2.5,3,3.5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]

    if (grade_mode == "fair"):
        num_f=num*5
        return get_region_value_f(num_f)
    elif (grade_mode == "easy"):
        if (num >=0.85):
            return 5
        elif ( 0.85>num>=0.8):
            return 4.5
        elif ( 0.8>num>=0.7):
            return 4
        else:
            num_e = num*5
            return get_region_value_e(num_e)
    elif (grade_mode == "lose_ends"):
        if (num >=0.85):
            return 5
        elif ( 0.85>num>=0.8):
            return 4.5
        elif ( 0.8>num>=0.7):
            return 4
        elif (0.2>=num):
            return 0
        else:
            num_l = num*5
            return get_region_value_l(num_l)
    else:
        print("not a valid mode ")


In [201]:
from sentence_transformers import util

In [202]:
from scipy.stats import spearmanr
from sklearn.metrics import r2_score

In [203]:
def Evaluation(embeddings_1=None, embeddings_2=None,test_data=None,mode="easy"):
    """
    this function takes two lists of embeddings in the form of torch tensor or list or df column one for the student answer and other for model answer
    or any two sentences if this function used in any other code and the test data frame and the mode of Gradazier and output MAE % and r_squared and 
    correlation_coefficient

    parameters
    embeddings_1:list or tensor or df column that hold the sentence embeddings (student answer embeddings)
    embeddings_2:list or tensor or df column that hold the sentence embeddings (model answer embeddings)
    data: the test data frame
    mode: the mode of which gradazier is used ["fair", "easy", "lose_ends"]
    """
    from sentence_transformers import util
    if (len(embeddings_1) != len(embeddings_2)):
        print("embeddings_1 and embeddings_2 are not the same length")
        return None
    elif (mode not in ["easy","lose_ends","fair"]):
        print("not valid mode")
        return None
    else:
        predicted = []
        for i in range(len(embeddings_1)):
            predicted.append(Gradanizer(util.cos_sim(embeddings_1[i], embeddings_2[i]),mode))
        transformed_grade=[]
        for i in range(len(embeddings_1)):
            transformed_grade.append(Gradanizer(test_data["grade"][i],mode))
            #transformed_grade.append(test_data["grade"][i]*5)
        arr_predicted =np.array(predicted)
        arr_grade=np.array(transformed_grade)
        MAE = (1-((np.sum(np.abs(arr_predicted-arr_grade))/len(embeddings_1))/5))*100
        correlation_coefficient, p_value = spearmanr(arr_grade, arr_predicted)
        r_squared = r2_score(arr_grade, arr_predicted)
        dif =arr_predicted-arr_grade
        hits_precentage= (np.count_nonzero(dif == 0)/len(embeddings_1))
        return MAE,correlation_coefficient,r_squared
        #if you want the hit precentage tag the above and untag the blew
        #return MAE,correlation_coefficient,hits_precentage
    

In [204]:
# Get embeddings for train and test data using different models
train_word2vec = test_data['student_answer'].apply(lambda x: get_embeddings(word2vec_model, x))
test_word2vec = test_data['model_answer'].apply(lambda x: get_embeddings(word2vec_model, x))


In [205]:
train_word2vec[0:5]

0    [0.038584497, -0.017718472, 0.053652238, 0.100...
1    [-0.0021623883, -0.0023314613, 0.052162714, 0....
2    [0.013964201, 0.008574185, 0.01831376, 0.06740...
3    [-0.009379069, 0.008200905, 0.059607305, 0.096...
4    [0.029686783, -0.0011798522, 0.0065001543, 0.0...
Name: student_answer, dtype: object

In [206]:
train_word2vec= torch.tensor(train_word2vec)
train_word2vec[0:5]

tensor([[ 0.0386, -0.0177,  0.0537,  ..., -0.0176,  0.0411,  0.0087],
        [-0.0022, -0.0023,  0.0522,  ..., -0.0582,  0.0067,  0.0026],
        [ 0.0140,  0.0086,  0.0183,  ..., -0.0259, -0.0333,  0.0174],
        [-0.0094,  0.0082,  0.0596,  ..., -0.0354,  0.0174, -0.0468],
        [ 0.0297, -0.0012,  0.0065,  ..., -0.0396,  0.0045, -0.0050]],
       dtype=torch.float64)

In [207]:
test_word2vec= torch.tensor(test_word2vec)
test_word2vec[0:5]

tensor([[ 0.0603, -0.0067,  0.0547,  ..., -0.0294,  0.0533, -0.0027],
        [-0.0121, -0.0123,  0.0270,  ..., -0.0526, -0.0025,  0.0232],
        [ 0.0164,  0.0048, -0.0057,  ..., -0.0028,  0.0467, -0.0071],
        [ 0.0603, -0.0067,  0.0547,  ..., -0.0294,  0.0533, -0.0027],
        [-0.0091,  0.0118,  0.0255,  ..., -0.0364, -0.0146, -0.0007]],
       dtype=torch.float64)

In [208]:
precentage,correlation_coefficient,r_squared = Evaluation(train_word2vec,test_word2vec,test_data,"fair")
print(precentage,correlation_coefficient,r_squared)

75.67338282078472 0.20310990847400123 -0.2531905664319225


In [209]:
precentage,correlation_coefficient,r_squared = Evaluation(train_word2vec,test_word2vec,test_data,"easy")
print(precentage,correlation_coefficient,r_squared)

74.60233297985154 0.21395528365860808 -0.26623965555920215


In [210]:
precentage,correlation_coefficient,r_squared = Evaluation(train_word2vec,test_word2vec,test_data,"lose_ends")
print(precentage,correlation_coefficient,r_squared)

71.89819724284199 0.2121318116148784 -0.24400963653160956


In [211]:
train_glove = np.array([nlp(text).vector for text in test_data['student_answer']])
test_glove = np.array([nlp(text).vector for text in test_data['model_answer']])


In [212]:
train_glove= torch.tensor(train_glove)
test_glove= torch.tensor(test_glove)

In [213]:
precentage,correlation_coefficient,r_squared = Evaluation(train_glove,test_glove,test_data,"fair")
print(precentage,correlation_coefficient,r_squared)

73.42523860021208 0.06520601135437996 -0.5530007747644257


In [214]:
precentage,correlation_coefficient,r_squared = Evaluation(train_glove,test_glove,test_data,"easy")
print(precentage,correlation_coefficient,r_squared)

73.90243902439025 0.04361425359018988 -0.5487960826142768


In [215]:
precentage,correlation_coefficient,r_squared = Evaluation(train_glove,test_glove,test_data,"lose_ends")
print(precentage,correlation_coefficient,r_squared)

71.62248144220572 0.0418207984264587 -0.5161487988798423


In [216]:
#fasttext.util.download_model('en', if_exists='ignore')  # Uncomment if you want to download the FastText model
#fasttext_model = fasttext.load_model('cc.en.300.bin')  # Load your FastText model (change the path if needed)


In [217]:

#train_fasttext = np.array([fasttext_model.get_sentence_vector(text) for text in test_data['student_answer']])
#test_fasttext = np.array([fasttext_model.get_sentence_vector(text) for text in test_data['model_answer']])


In [218]:
#test_fasttext= torch.tensor(test_fasttext)
#train_fasttext= torch.tensor(train_fasttext)

In [219]:
#!pip install conceptnet

#from conceptnet.nodes import standardized_concept_uri
#from conceptnet.vectors.query import VectorSpaceWrapper

In [221]:
# Train the initial LSA model on the first dataset
initial_text_data = train_data_analysis['student_answer'].tolist() + train_data_analysis['model_answer'].tolist()
initial_tfidf_matrix = TfidfVectorizer().fit_transform(initial_text_data)
initial_lsa_model = TruncatedSVD(n_components=11).fit(initial_tfidf_matrix)

# Store the initial LSA model and TF-IDF matrix
overall_embedding_matrix = initial_lsa_model.transform(initial_tfidf_matrix)
overall_tfidf_matrix = initial_tfidf_matrix


In [240]:
# Combine student and model answers
all_answers = train_data_analysis['student_answer'].tolist() + train_data_analysis['model_answer'].tolist()

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_answers)

# Perform Singular Value Decomposition (LSA)
num_dimensions = 11  # Choose the number of dimensions for the embeddings
lsa = TruncatedSVD(n_components=num_dimensions)
embedding_matrix = lsa.fit_transform(tfidf_matrix)


In [241]:
# Combine student and model answers
all_answers2 = test_data['student_answer'].tolist() + test_data['model_answer'].tolist()

tfidf_matrix = vectorizer.transform(all_answers2)

embedding_matrix = lsa.transform(tfidf_matrix)

# Separate embeddings for student and model answers
student_embeddings2 = embedding_matrix[:len(test_data['student_answer'])]
model_embeddings2 = embedding_matrix[len(test_data['student_answer']):]

In [242]:
precentage,correlation_coefficient,r_squared = Evaluation(student_embeddings2,student_embeddings2,test_data,"fair")
print(precentage,correlation_coefficient,r_squared)

70.95440084835631 0.09677187203405786 -1.05425016067396


In [243]:
precentage,correlation_coefficient,r_squared = Evaluation(student_embeddings2,student_embeddings2,test_data,"easy")
print(precentage,correlation_coefficient,r_squared)

73.71155885471899 0.09314136286388634 -0.7897280793406143


In [244]:
precentage,correlation_coefficient,r_squared = Evaluation(student_embeddings2,student_embeddings2,test_data,"lose_ends")
print(precentage,correlation_coefficient,r_squared)

71.5694591728526 0.09011019129467626 -0.7251231308460437


In [246]:
# Combine student and model answers
all_answers = train_data_644['student_answer'].tolist() + train_data_644['model_answer'].tolist()


tfidf_matrix = vectorizer.fit_transform(all_answers)

embedding_matrix = lsa.fit(tfidf_matrix)


In [247]:
# Combine student and model answers
all_answers2 = test_data['student_answer'].tolist() + test_data['model_answer'].tolist()

tfidf_matrix = vectorizer.transform(all_answers2)

embedding_matrix = lsa.transform(tfidf_matrix)

# Separate embeddings for student and model answers
student_embeddings2 = embedding_matrix[:len(test_data['student_answer'])]
model_embeddings2 = embedding_matrix[len(test_data['student_answer']):]

In [248]:
precentage,correlation_coefficient,r_squared = Evaluation(student_embeddings2,student_embeddings2,test_data,"fair")
print(precentage,correlation_coefficient,r_squared)

70.53022269353129 0.07340128025859846 -1.1079314871435217


In [249]:
# Combine student and model answers
all_answers = train_data_translated['student_answer'].tolist() + train_data_translated['model_answer'].tolist()


tfidf_matrix = vectorizer.fit_transform(all_answers)

embedding_matrix = lsa.fit(tfidf_matrix)


In [250]:
# Combine student and model answers
all_answers2 = test_data['student_answer'].tolist() + test_data['model_answer'].tolist()

tfidf_matrix = vectorizer.transform(all_answers2)

embedding_matrix = lsa.transform(tfidf_matrix)

# Separate embeddings for student and model answers
student_embeddings2 = embedding_matrix[:len(test_data['student_answer'])]
model_embeddings2 = embedding_matrix[len(test_data['student_answer']):]

In [251]:
precentage,correlation_coefficient,r_squared = Evaluation(student_embeddings2,student_embeddings2,test_data,"fair")
print(precentage,correlation_coefficient,r_squared)

70.6574761399788 0.01756348328179278 -1.0918270892026531
